In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Explore data

* **Note** we note that keyword is not a real good classifier to obtain our target, and also we note that location can be an important feature to add with the text to predict our target.

# Preprocessing data

## Missing Values
### Missing keywords

In [45]:
import pandas as pd
import importlib 
import preprocess.keywords as kw
importlib.reload(kw)
layer=kw.PreprocessKeywords()
layer.preprocess(train_data)


,id,keyword,location,text,target,keyword_prep
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Reason
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,fire
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,evacuation
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,evacuation
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,smoke
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,collapse
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,.
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,Hawaii
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,collided


## Preprocessing location

In [7]:
train_data = train_data.dropna(subset=['location'])
train_data

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [10]:
import preprocess.location as pl
from geopy.geocoders import Nominatim
from typing import Tuple,Any
featuretest_data= train_data.copy()

class ConversionProgress(pl.GeolocationStepConverter):
    def __init__(self,frequency:int):
        self.frequency=frequency
        self.counter=0
        self.progress=0
    def convert(self, address:str)-> Tuple[Any,bool]:
        self.progress+=1
        self.counter+=1
        if self.counter >= self.frequency:
            self.counter=0
            print(f"Current progress: {self.progress}")
        return address,False    
        


In [19]:
geoLoc = Nominatim(user_agent="GetLoc")
rateLimited = pl.RateLimitedGeocoder(geoLoc, min_delay_seconds=1)
layer=pl.DefaultPreprocessLocationLayer(force_update=False)
layer.transformers.append(ConversionProgress(frequency=100))
layer.transformers.append(pl.StripLocation())
layer.transformers.append(pl.LatLongToGeolocation(rateLimited))
layer.transformers.append(pl.AddressToGeolocation(rateLimited))
featuretest_data=layer.preprocess(featuretest_data)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Livingston, IL  U.S.A.',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 1390, in getresponse
    response.begin()
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 286, in _read_status
    line = str

Current progress: 100
Current progress: 200


RateLimiter caught an error, retrying (0/2 tries). Called with (*('?s????ss? a?????',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 1390, in getresponse
    response.begin()
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/home/ramizouari/anaconda3/envs/nlp/lib/python3.11/http/client.py", line 286, in _read_status
    line = str(self.

Current progress: 300


KeyboardInterrupt: 

In [32]:
featuretest_data.to_pickle("location.pkl")

NameError: name 'featuretest_data' is not defined

In [20]:
featuretest_data=pd.read_pickle("location.pkl")

In [21]:
featuretest_data["

,id,keyword,location,text,target,geolocation,geolocation_converted
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,"(Birmingham, West Midlands Combined Authority,...",True
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,NaN,False
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,"(Africa, (11.5024338, 17.7578122))",True
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,"(Philadelphia, Philadelphia County, Pennsylvan...",True
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,"(London, Greater London, England, United Kingd...",True
...,...,...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0,"(تونس, (33.8439408, 9.400138))",True
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0,NaN,False
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,"(Vancouver, Metro Vancouver Regional District,...",True
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,"(London, Greater London, England, United Kingd...",True


In [1]:
import pandas as pd

C:\Users\myass\AppData\Local\Temp\ipykernel_11144\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
